In [1]:
import numpy as np
import pandas as pd
import os

# Set current working directory
Current = os.getcwd()
os.chdir(Current)

# Load the new dataset
rainfall_data = pd.read_csv('Prefilled-RainfallData_Raw_SWRB.csv', index_col='Date', parse_dates=True)

# Convert columns to numeric
for col in rainfall_data.columns:
    rainfall_data[col] = pd.to_numeric(rainfall_data[col], errors='coerce')

rainfall_data = rainfall_data.reset_index()
o1 = rainfall_data.copy()
filled = rainfall_data.copy()
o2 = o1.fillna(0)  # fill the empty data with zero

# Insert columns for Year, Month, Day
rainfall_data.insert(1, "Year", rainfall_data['Date'].dt.year)
rainfall_data.insert(2, "Month", rainfall_data['Date'].dt.strftime('%b'))
rainfall_data.insert(3, "Day", rainfall_data['Date'].dt.day)

# Calculation of long-term daily average
ltda = pd.pivot_table(rainfall_data, index=['Month', 'Day'], aggfunc='mean', values=rainfall_data.columns[4:], sort=False)
ltda.reset_index(inplace=True)
ltda.insert(0, "Index", ltda['Month'].map(str) + '-' + ltda['Day'].map(str))
ltda = ltda.drop(columns=['Month', 'Day'])
ltda.set_index("Index", inplace=True)

# Count data in each year and month
count = pd.pivot_table(rainfall_data, index=['Year'], aggfunc='count', values=rainfall_data.columns[4:])
count[(count == 0)] = pd.NA

count2 = pd.pivot_table(rainfall_data, index=['Year', 'Month'], aggfunc='count', values=rainfall_data.columns[4:], sort=False)
count2[(count2 == 0)] = pd.NA
count2.reset_index(inplace=True)

# Calculation of annual sum of rainfall
asr = pd.pivot_table(rainfall_data, index=['Year'], aggfunc=['sum', 'count'], values=rainfall_data.columns[4:])
mask = (asr.xs('count', axis=1) == 0)
asr = asr.xs('sum', axis=1)
asr[mask] = pd.NA
asr = asr.reset_index()

# Calculate yearly average and count
yr_avg = pd.DataFrame(asr.mean(axis=0)).T
yr_avg.index = ['average']
yr_count = pd.DataFrame(asr.count()).T
yr_count.index = ['count']
asr = pd.concat([asr, yr_avg, yr_count])

# Calculation of average long-term monthly sum of rainfall
altm = pd.pivot_table(rainfall_data, index=['Year', 'Month'], aggfunc=['sum', 'count'], values=rainfall_data.columns[4:], sort=False)
mask = (altm.xs('count', axis=1) == 0)
altm = altm.xs('sum', axis=1)
altm[mask] = pd.NA
altm.reset_index(inplace=True)
monthly = pd.pivot_table(altm, index=['Month'], aggfunc='mean', values=altm.columns[2:], sort=False)

# Calculation of normal rainfall
countnr = pd.pivot_table(rainfall_data, index=['Year', 'Month'], aggfunc='count', values=rainfall_data.columns[4:], sort=False)
count_year = pd.pivot_table(rainfall_data, index=['Year'], aggfunc='count', values=rainfall_data.columns[4:], sort=False)
normal_rain = pd.pivot_table(rainfall_data, index=['Year'], aggfunc=sum, values=rainfall_data.columns[4:], sort=False)
normal_rain[(count_year == 0)] = pd.NA

mask1 = pd.pivot_table(countnr.query('Month==["Jun","Jul","Aug","Sep"]'), index='Year', aggfunc=sum, values=countnr.columns, sort=False)
normal_rain[(mask1 < 122)] = pd.NA

mask2 = pd.pivot_table(countnr.query('Month==["May","Oct","Nov","Dec"]'), index='Year', aggfunc=sum, values=countnr.columns, sort=False)
normal_rain[(mask2 < 114)] = pd.NA

normal_rain.replace(0, pd.NA, inplace=True)
yr_avg = pd.DataFrame(normal_rain.mean(axis=0)).T
yr_avg.index = ['average']
yr_count = pd.DataFrame(normal_rain.count()).T
yr_count.index = ['count']
normal_rain = pd.concat([normal_rain, yr_avg])
normal_rain = pd.concat([normal_rain, yr_count])

# Normal Annual Rainfall (NAR)
NAR = normal_rain.drop(index=['average', 'count']).mean(axis=0)
NAR = pd.DataFrame(NAR, columns=['NormalAnnualRainfall'])
NAR.index.name = 'Station'
NAR.to_csv('NAR.csv')

# Load station matrix
#original_matrix = pd.read_excel('/mnt/data/stations_matrix.xlsx')

# Generate station matrix for the new data
stations = rainfall_data.columns[4:]
station_matrix = pd.DataFrame(index=stations, columns=['Target'] + [f'Reference_{i}' for i in range(1, len(stations))])

for i, station in enumerate(stations):
    station_matrix.at[station, 'Target'] = station
    references = [s for s in stations if s != station]
    for j, ref in enumerate(references[:len(station_matrix.columns) - 1]):
        station_matrix.at[station, f'Reference_{j+1}'] = ref

# Save the new station matrix
station_matrix.to_excel('generated_stations_matrix.xlsx', index=False)

# Dictionary to store errors
station_errors = {station: {'MAE': [], 'RMSE': []} for station in stations}

In [4]:
# Perform LOOCV
for i in range(len(station_matrix)):
    target_station = str(station_matrix.iloc[i, 0])
    tar_idx = NAR.index.tolist().index(target_station)
    row_list = station_matrix.iloc[[i]].dropna(axis=1)
    ref_stations = [str(x) for x in row_list.values.tolist()[0][1:]]
    ref_indices = [filled.columns.tolist().index(ref) for ref in ref_stations]
    
    print(f"\nProcessing station: {target_station} ({i + 1}/{len(station_matrix)})")
    
    for j in range(len(filled)):
        if pd.notna(o1.loc[j, target_station]):
            original_value = o1.loc[j, target_station]
            
            # Temporarily remove the observed value
            o1.loc[j, target_station] = np.nan
            
            if o1.loc[j, ref_stations].count() > 0:
                store = 0
                for k in range(len(ref_stations)):
                    if j < len(o2) and ref_stations[k] in o2.columns:
                        store += o2.loc[j, ref_stations[k]] / NAR.loc[ref_stations[k], 'NormalAnnualRainfall']
                estimated_value = NAR.loc[target_station, 'NormalAnnualRainfall'] * store / o1.loc[j, ref_stations].count()
                filled.loc[j, target_station] = estimated_value
                
                # Calculate errors for a single data point
                mae = abs(original_value - estimated_value)
                rmse = (original_value - estimated_value) ** 2

                # For overall error calculation, store the squared error
                station_errors[target_station]['MAE'].append(mae)
                station_errors[target_station]['RMSE'].append(rmse)
                
                #print(f"  Data point {j + 1}/{len(filled)}: MAE={mae:.4f}, RMSE={rmse:.4f}")
            
            # Restore the original value
            o1.loc[j, target_station] = original_value


Processing station: 411 (1/9)

Processing station: 412 (2/9)

Processing station: 414 (3/9)

Processing station: 416 (4/9)

Processing station: 417 (5/9)

Processing station: 419 (6/9)

Processing station: 420 (7/9)

Processing station: 501 (8/9)

Processing station: 502 (9/9)


In [5]:
# Calculate overall MAE and RMSE for each station
for station in station_errors:
    station_errors[station]['MAE'] = np.mean(station_errors[station]['MAE'])
    station_errors[station]['RMSE'] = np.sqrt(np.mean(station_errors[station]['RMSE']))

# Calculate overall MAE and RMSE across all stations
all_mae = np.mean([station_errors[station]['MAE'] for station in station_errors])
all_rmse = np.mean([station_errors[station]['RMSE'] for station in station_errors])

print("Station Errors:")
print(station_errors)
print("\nOverall MAE:", all_mae)
print("Overall RMSE:", all_rmse)

Station Errors:
{'411': {'MAE': 5.27732207014938, 'RMSE': 15.291896417417687}, '412': {'MAE': 4.442456802958206, 'RMSE': 12.200551813022301}, '414': {'MAE': 4.336079298172702, 'RMSE': 10.835540070286395}, '416': {'MAE': 3.998597921939488, 'RMSE': 11.452396174216345}, '417': {'MAE': 4.564939732450066, 'RMSE': 12.43899266246042}, '419': {'MAE': 4.447133285056028, 'RMSE': 11.655695674843617}, '420': {'MAE': 4.034448028532151, 'RMSE': 10.870571980707217}, '501': {'MAE': 5.896502006278916, 'RMSE': 13.523657352721365}, '502': {'MAE': 4.154540982267893, 'RMSE': 9.318228336279804}}

Overall MAE: 4.572446680867204
Overall RMSE: 11.954170053550571
